In [2]:
import numpy as np
import pandas as pd

def simulate_hcp_brick_poisson(
    n_hcps=400,
    n_bricks=80,
    avg_bricks_per_hcp=2.5,
    avg_hcps_per_brick=18,
    seed=7,
):
    rng = np.random.default_rng(seed)

    # ----- (A) HCP features -----
    specialties = np.array(["GP", "CARDIO", "ENDO", "ONCO", "PULMO"])
    spec = rng.choice(specialties, size=n_hcps, p=[0.45, 0.18, 0.15, 0.12, 0.10])

    seniority = rng.integers(0, 21, size=n_hcps)  # years (0..20)
    potential = rng.lognormal(mean=0.0, sigma=0.6, size=n_hcps)  # positive continuous

    # One-hot for specialty
    spec_df = pd.get_dummies(spec, prefix="spec")
    X = np.column_stack([
        np.ones(n_hcps),                         # intercept
        seniority / 10.0,                        # scaled
        np.log1p(potential),                     # log feature
        spec_df.to_numpy(),                      # dummies
    ])
    feature_names = (["intercept", "seniority_x0.1", "log1p_potential"]
                     + list(spec_df.columns))

    p = X.shape[1]

    # ----- (B) True beta* (ground truth) -----
    beta_true = np.zeros(p)
    beta_true[0] = -0.3                    # intercept
    beta_true[1] =  0.20                   # seniority helps
    beta_true[2] =  0.55                   # potential helps

    # Specialty effects (relative to baseline; here baseline is implicitly captured by intercept)
    # Put mild lifts / penalties
    for j, name in enumerate(feature_names):
        if name == "spec_CARDIO":
            beta_true[j] = 0.25
        elif name == "spec_ONCO":
            beta_true[j] = 0.35
        elif name == "spec_PULMO":
            beta_true[j] = 0.10
        elif name == "spec_ENDO":
            beta_true[j] = 0.15
        # GP left at 0 shift (reference-ish), though with full one-hot it's not a strict reference.
    # (With full one-hot + intercept, there is collinearity; in practice you drop one dummy.
    # For simulation it's fine; for fitting, use a reference category or regularize.)

    # True productivity per HCP
    lambda_true = np.exp(X @ beta_true)

    # ----- (C) Brick membership + exposure w_{h,b} -----
    # Goal: heterogeneous bricks + overlaps.
    # We'll assign each HCP to K bricks, K ~ Poisson(avg_bricks_per_hcp) truncated >=1
    K = rng.poisson(lam=avg_bricks_per_hcp, size=n_hcps)
    K = np.clip(K, 1, max(1, n_bricks // 2))

    # Popularity of bricks (some bricks are "bigger")
    brick_pop = rng.lognormal(mean=0.0, sigma=0.7, size=n_bricks)
    brick_pop = brick_pop / brick_pop.sum()

    # Build sparse membership list
    rows = []
    for h in range(n_hcps):
        bricks_h = rng.choice(n_bricks, size=K[h], replace=False, p=brick_pop)
        # visits per (h,b): more variable, depends on seniority & random
        # visits = base + seniority effect + noise, then clipped >=0
        base = rng.poisson(lam=2.5)  # baseline
        for b in bricks_h:
            visits = base + rng.poisson(lam=0.15 * seniority[h] + 1.0)
            # exposure: use visits directly (or normalized variant)
            w_hb = float(visits)
            if w_hb == 0:
                w_hb = 0.5  # keep small exposure if assigned (optional)
            rows.append((h, b, visits, w_hb))

    hb = pd.DataFrame(rows, columns=["hcp_id", "brick_id", "visits", "w"])

    # Make bricks heterogeneous: optionally drop/keep to target avg_hcps_per_brick
    # We'll just report achieved distribution; simulation already creates variation.

    # ----- (D) Generate latent z* and observed y -----
    # z_{h,b} ~ Poisson(lambda_h * w_{h,b})
    mu = lambda_true[hb["hcp_id"].to_numpy()] * hb["w"].to_numpy()
    z_true = rng.poisson(lam=mu)

    hb["z_true"] = z_true

    # Aggregate observed sales by brick
    y_by_brick = hb.groupby("brick_id")["z_true"].sum().rename("y").reset_index()

    # Pack HCP table
    hcp = pd.DataFrame({
        "hcp_id": np.arange(n_hcps),
        "specialty": spec,
        "seniority": seniority,
        "potential": potential,
        "lambda_true": lambda_true,
    })
    for j, nm in enumerate(feature_names):
        hcp[nm] = X[:, j]

    meta = {
        "feature_names": feature_names,
        "beta_true": beta_true,
        "n_hcps": n_hcps,
        "n_bricks": n_bricks,
    }

    return hcp, hb, y_by_brick, meta

Qué te devuelve

- `hcp_df`: tabla HCP con features y `lambda_true`
- `hb_df`: tabla de relaciones HCP–brick con `w`, v`isits, y **ground truth** `z_true`
- `y_df`: ventas observadas por brick (lo único que normalmente tendrías)
- `meta`: `beta_true` y nombres de features

Perfecto — vamos **columna por columna**, con **significado estadístico** y **rol en el modelo**.
Me centraré en `hcp_df`, que representa el **nivel HCP** (unidad individual).

---

# 📊 `hcp_df` — explicación detallada de cada columna

`hcp_df` contiene **una fila por HCP**.
Conceptualmente mezcla **datos observables** + **verdad simulada** (solo para evaluación).

---

## 🆔 Identificación

### **`hcp_id`**

* Identificador único del HCP
* Tipo: entero
* Uso:

  * Clave para unir con `hb_df`
  * Nunca entra directamente al modelo

---

## 🧠 Variables “reales” (observables en la vida real)

Estas son las **features que sí existirían en producción**.

---

### **`specialty`**

* Tipo: categórica (`GP`, `CARDIO`, `ENDO`, `ONCO`, `PULMO`)
* Significado:

  * Especialidad médica del HCP
* Uso:

  * Se transforma a *one-hot* (ver columnas `spec_*`)
  * Captura diferencias estructurales de productividad

📌 En el modelo:

> Diferentes especialidades → diferente productividad esperada

---

### **`seniority`**

* Tipo: entero (0–20)
* Significado:

  * Años de experiencia del HCP
* Uso:

  * Feature numérica
  * Se escala (`seniority / 10`) para estabilidad numérica

📌 Interpretación:

> Cada +10 años de seniority multiplica la tasa por `exp(β_seniority)`

---

### **`potential`**

* Tipo: continuo positivo (lognormal)
* Significado:

  * Proxy de “potencial comercial” del HCP
* Uso:

  * Se transforma como `log1p(potential)`
  * Reduce skew y hace la relación más lineal en log-space

📌 Interpretación:

> HCPs con mayor potencial generan más ventas esperadas

---

## 🧮 Variables de diseño (matriz (X_h))

Estas columnas **sí entran directamente** en:

[
\lambda_h = \exp(X_h \beta)
]

---

### **`intercept`**

* Siempre vale 1
* Significado:

  * Nivel base de productividad
* Uso:

  * Captura la tasa media cuando todas las demás features son 0

📌 Sin intercept:

* el modelo estaría forzado a pasar por el origen
* mala idea en Poisson / Gamma

---

### **`seniority_x0.1`**

* Definición: `seniority / 10`
* Tipo: continuo
* Significado:

  * Versión escalada de seniority
* Uso:

  * Mejora convergencia del GLM
  * Hace β más interpretable

📌 Interpretación de β:

> +0.2 ⇒ +10 años → ~+22% en tasa (exp(0.2))

---

### **`log1p_potential`**

* Definición: `log(1 + potential)`
* Tipo: continuo
* Significado:

  * Transformación suave del potencial
* Uso:

  * Evita que outliers dominen el ajuste

📌 Interpretación:

> Relación multiplicativa y estable con la productividad

---

### **`spec_GP`, `spec_CARDIO`, `spec_ENDO`, `spec_ONCO`, `spec_PULMO`**

* Tipo: binarias (0/1)
* Significado:

  * One-hot encoding de la especialidad
* Uso:

  * Permite efectos específicos por especialidad

⚠️ Nota técnica importante:

* Con **intercept + todas las dummies** hay colinealidad perfecta
* En producción harías una de estas:

  * eliminar una dummy (categoría de referencia)
  * o usar regularización (ridge)
* En la simulación se tolera porque:

  * queremos control total del ground truth
  * EM + ridge lo maneja bien

📌 Interpretación:

> β positivo ⇒ esa especialidad es más productiva que la media

---

## 🔮 Variables de “ground truth” (NO observables en la vida real)

Estas columnas **solo existen para evaluación** del método.

---

### **`lambda_true`**

* Tipo: continuo positivo
* Definición:
  [
  \lambda_h^{*} = \exp(X_h \beta^{*})
  ]
* Significado:

  * Productividad real subyacente del HCP
* Uso:

  * Comparar con (\hat\lambda_h) estimado por EM
  * Diagnóstico de sesgo / varianza

📌 Nunca estaría disponible en datos reales.

---

## 🧩 Resumen por niveles

### 🔹 Observables reales

| Columna     | Uso                |
| ----------- | ------------------ |
| `hcp_id`    | Identificación     |
| `specialty` | Feature categórica |
| `seniority` | Feature numérica   |
| `potential` | Feature numérica   |

---

### 🔹 Diseño del modelo

| Columna           | Entra en (X_h)? |
| ----------------- | --------------- |
| `intercept`       | ✔️              |
| `seniority_x0.1`  | ✔️              |
| `log1p_potential` | ✔️              |
| `spec_*`          | ✔️              |

---

### 🔹 Solo para evaluación

| Columna       | En producción |
| ------------- | ------------- |
| `lambda_true` | ❌             |

---

## 🧠 Lectura mental correcta

> `hcp_df` define **quién es el HCP** y **qué esperamos de él en promedio**.
> El EM **no aprende ventas por HCP directamente**,
> aprende **cómo esas features explican ventas agregadas por brick**.

---

In [3]:
# Example usage
hcp_df, hb_df, y_df, meta = simulate_hcp_brick_poisson(seed=42)
print(hcp_df.head())

   hcp_id specialty  seniority  potential  lambda_true  intercept  \
0       0      ENDO          0   2.114398     1.607724        1.0   
1       1        GP          6   0.859409     1.174852        1.0   
2       2      ONCO         10   1.243677     2.002635        1.0   
3       3      ENDO         19   0.235521     1.413857        1.0   
4       4        GP         15   0.499669     1.249676        1.0   

   seniority_x0.1  log1p_potential  spec_CARDIO  spec_ENDO  spec_GP  \
0             0.0         1.136036          0.0        1.0      0.0   
1             0.6         0.620259          0.0        0.0      1.0   
2             1.0         0.808116          0.0        0.0      0.0   
3             1.9         0.211493          0.0        1.0      0.0   
4             1.5         0.405245          0.0        0.0      1.0   

   spec_ONCO  spec_PULMO  
0        0.0         0.0  
1        0.0         0.0  
2        1.0         0.0  
3        0.0         0.0  
4        0.0         0.

Perfecto. Vamos **igual de sistemático** con las otras dos tablas.
Primero `hb_df` (nivel HCP–Brick), luego `y_df` (nivel Brick).

---

# 📊 `hb_df` — tabla HCP–Brick (el corazón del problema)

`hb_df` está en formato **long** y representa **una relación HCP–Brick**.
Aquí viven **las variables latentes y la exposición**.

---

## 🆔 Identificación y joins

### **`hcp_id`**

* Identificador del HCP
* FK hacia `hcp_df.hcp_id`
* Uso:

  * unir features (X_h)
  * agrupar resultados por HCP

---

### **`brick_id`**

* Identificador del brick
* FK hacia `y_df.brick_id`
* Uso:

  * agrupar ventas observadas
  * definir los conjuntos (\mathcal{H}_b)

---

## 📈 Variables de exposición (observables reales)

Estas columnas **sí existirían en producción**.

---

### **`visits`**

* Tipo: entero ≥ 0
* Significado:

  * Número de visitas del HCP al brick
* Uso:

  * Proxy de oportunidad / esfuerzo
  * Puede entrar:

    * directamente como `w_{h,b}`
    * o como feature adicional en (X_h) o a nivel interacción

📌 Intuición:

> A más visitas → mayor exposición → más ventas esperadas

---

### **`w`**

* Tipo: continuo positivo
* Significado:

  * Exposición efectiva del HCP (h) en el brick (b)
* Uso en el modelo:
  [
  z_{h,b} \sim \text{Poisson}(\lambda_h , w_{h,b})
  ]

En la simulación:

* `w ≈ visits`
* Con un pequeño floor para evitar ceros

📌 En producción:

* `w` puede ser:

  * visitas
  * share de territorio
  * tiempo dedicado
  * combinación ponderada

---

## 👻 Variables latentes (ground truth solo para evaluación)

---

### **`z_true`**

* Tipo: entero ≥ 0
* Significado:

  * Ventas *reales* atribuibles al HCP (h) en el brick (b)
* Generación:
  [
  z_{h,b}^* \sim \text{Poisson}(\lambda_h^* w_{h,b})
  ]

📌 En datos reales:

* ❌ nunca observable
* ✔️ es lo que EM intenta recuperar **en esperanza**

---

## 🧠 Rol conceptual de `hb_df`

Esta tabla:

* Define **quién compite con quién** dentro de cada brick
* Define las **restricciones estructurales**:
  [
  y_b = \sum_{h\in\mathcal{H}*b} z*{h,b}
  ]
* Es donde vive el **E-step**

---

## 🧩 Resumen `hb_df`

| Columna    | Nivel     | En producción | Rol                     |
| ---------- | --------- | ------------- | ----------------------- |
| `hcp_id`   | ID        | ✔️            | Join                    |
| `brick_id` | ID        | ✔️            | Join                    |
| `visits`   | Observada | ✔️            | Proxy esfuerzo          |
| `w`        | Observada | ✔️            | Exposición en el modelo |
| `z_true`   | Latente   | ❌             | Ground truth            |

---

In [4]:
print(hb_df.head())

   hcp_id  brick_id  visits    w  z_true
0       0        45       5  5.0       7
1       0         1       3  3.0       1
2       0        59       2  2.0       4
3       0        76       2  2.0       8
4       1        18       3  3.0       5



---

# 📊 `y_df` — tabla de ventas por Brick

`y_df` representa **exactamente lo que tienes en la vida real**.

---

## 🆔 Identificación

### **`brick_id`**

* Identificador único del brick
* Uso:

  * define la unidad de observación
  * join con `hb_df`

---

## 💰 Variable objetivo observada

---

### **`y`**

* Tipo: entero ≥ 0
* Significado:

  * Ventas totales del brick
* Definición:
  [
  y_b = \sum_{h\in\mathcal{H}*b} z*{h,b}
  ]

📌 Este es el **único target real** del problema.

---

## 🧠 Rol conceptual de `y_df`

* Es la **likelihood observada**
* EM intenta explicar estos (y_b) usando:

  * estructura de bricks (`hb_df`)
  * productividad HCP (`hcp_df`)
* No hay labels por HCP → **problema inverso**

---

## 🧩 Resumen `y_df`

| Columna    | En producción | Rol              |
| ---------- | ------------- | ---------------- |
| `brick_id` | ✔️            | Identificador    |
| `y`        | ✔️            | Ventas agregadas |

---

In [5]:
print(y_df.head())

   brick_id    y
0         0  297
1         1  172
2         2  134
3         3   23
4         4  209



## 🔗 Cómo encajan las 3 tablas (lectura final)

```
hcp_df   →  define λ_h = exp(X_h β)
   ↓
hb_df    →  define dónde y cuánto se expone cada HCP (w_{h,b})
   ↓
y_df     →  observa solo la suma por brick
```

EM hace:

* **E-step**: trabaja en `hb_df` (estima ẑ)
* **M-step**: trabaja en `hcp_df` (actualiza β)
* **Likelihood**: vive en `y_df`

---

## ✅ Checkpoint final

Con esto ya tienes:

* modelo teórico ✔️
* datos mock ✔️
* interpretación de cada columna ✔️

In [7]:
print(meta)

{'feature_names': ['intercept', 'seniority_x0.1', 'log1p_potential', 'spec_CARDIO', 'spec_ENDO', 'spec_GP', 'spec_ONCO', 'spec_PULMO'], 'beta_true': array([-0.3 ,  0.2 ,  0.55,  0.25,  0.15,  0.  ,  0.35,  0.1 ]), 'n_hcps': 400, 'n_bricks': 80}


In [6]:
print("beta_true:", meta["beta_true"])

beta_true: [-0.3   0.2   0.55  0.25  0.15  0.    0.35  0.1 ]
